In [1]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from tensorflow.keras.optimizers import Adam


In [2]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [3]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    elif scale == 'YcrCb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    else:
        raise ValueError("Escala de cor não suportada.")

In [4]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

                # Converta para a escala de cor desejada
                img = convert_color_scale(img, color_scale)

                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

                images.append(img)
    return np.array(images)

In [5]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
normal_dir = os.path.join(data_dir, 'Non_Cancerous')
Serous= os.path.join(data_dir, 'Serous')
Mucinous = os.path.join(data_dir, 'Mucinous')
Endometrioid = os.path.join(data_dir, 'Endometri')
Clear = os.path.join(data_dir, 'Clear_Cell')
img_extensions = ['.jpg', '.jpeg', '.png']


In [6]:
# Carregar o modelo InceptionV3 pré-treinado
base_model = EfficientNetB0(weights='imagenet', include_top=False)     # <----------------------------------------------------------

# Extrair características usando o modelo VGG16
def extract_features(model, X):
    features = model.predict(X)
    features_flat = features.reshape((features.shape[0], -1))
    # np.savetxt("features_5Classes.csv", features_flat, delimiter=",")
    return features_flat


## <span style="color:green;">a partir daqui, complica a situação</span>

In [7]:
# Carregamento de imagens e conversão para XYZ
normal_images = load_images(normal_dir, 'grayscale',img_extensions)       
serous_images = load_images(Serous, 'grayscale',img_extensions)        
mucinous_images = load_images(Mucinous, 'grayscale',img_extensions)        
endometrioid_images = load_images(Endometrioid, 'grayscale',img_extensions)        
clear_images = load_images(Clear, 'grayscale',img_extensions)   


#      ISSO SERIA SO PRA 2 CLASSES
# Rótulos para imagens (0 para normal, 1 para câncer)
# normal_labels = np.zeros(normal_images.shape[0])
# cancer_labels = np.ones(cancer_images.shape[0])

# # Concatenar imagens e rótulos
# all_images = np.concatenate([normal_images, cancer_images], axis=0)
# all_images = extract_features(base_model, all_images)
# all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)


# Rótulos para imagens (0 para normal, 1 para seroso, 2 para mucinoso, 3 para endometrioide, 4 para células claras)
y = (
    ['Non_Cancerous'] * len(normal_images) +
    ['Serous'] * len(serous_images) +
    ['Mucinous'] * len(mucinous_images) +
    ['Endometrioid'] * len(endometrioid_images) +
    ['Clear_Cell'] * len(clear_images)
)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

all_images = np.concatenate([
    normal_images,
    serous_images,
    mucinous_images,
    endometrioid_images,
    clear_images
], axis=0)
all_images = extract_features(base_model, all_images)



# Dividi em treino e teste 80/20
X_train, X_test, y_train, y_test = train_test_split(all_images, y_encoded, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0


# testar ...

In [9]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Carregar os dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar os dados
x_train = x_train / 255.0
x_test = x_test / 255.0

# Converter os rótulos para one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Criar o modelo
model = Sequential([
    Flatten(input_shape=(28, 28)),         # transforma a imagem em vetor
    Dense(128, activation='relu'),         # camada oculta
    Dense(10, activation='softmax')        # saída para 10 classes (0 a 9)
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
model.fit(x_train, y_train, epochs=5, validation_split=0.1)

# Avaliar o modelo
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Acurácia no teste: {test_acc:.2f}")


11501568/11490434 [==============================] - 2s 0us/step
Epoch 1/5
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2803 - accuracy: 0.9187 - val_loss: 0.1272 - val_accuracy: 0.9657
Epoch 2/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1264 - accuracy: 0.9617 - val_loss: 0.0956 - val_accuracy: 0.9737
Epoch 3/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0863 - accuracy: 0.9744 - val_loss: 0.0861 - val_accuracy: 0.9762
Epoch 4/5
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0660 - accuracy: 0.9801 - val_loss: 0.0810 - val_accuracy: 0.9783
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0796 - accuracy: 0.9759
Acurácia no teste: 0.98
